In [2]:
import pandas as pd
import numpy as np
import math

# ======================
# Global Constants
# ======================
SERVICE_TIME        = 1.0           # phút phục vụ tại mỗi khách
TRUCK_SPEED         = 50.0 / 60.0   # đơn vị khoảng cách / phút (50 đv/h)
MAX_CAPACITY        = 500           # sức chứa tối đa mỗi xe
NEW_VEHICLE_PENALTY = 50.0          # (không dùng trong greedy nhưng giữ để đối chiếu)
PENALTY_FACTOR      = 100.0         # (không dùng trong greedy)
SERVED_BONUS        = 150.0         # (không dùng trong greedy)
DEMAND_BONUS        = 2.0           # (không dùng trong greedy)

# ======================
# Utils
# ======================
def euclidean_distance(x1,y1,x2,y2):
    return math.hypot(x1-x2, y1-y2)

# ======================
# Greedy Solver
# ======================
def greedy_dscvrptw(df, num_vehicles=6):
    N = len(df)
    served = set([0])              # depot index = 0
    unserved = set(range(1, N))
    total_vehicle_idle = 0.0
    total_customer_wait = 0.0

    # initialize 1 vehicle
    vehicles = [{
        'route': [0],
        'current_node': 0,
        'time': 0.0,
        'capacity': MAX_CAPACITY,
        'distance': 0.0
    }]

    def available_customers(t):
        return [i for i in unserved if df.at[i, 'time'] <= t]

    idx_v = 0
    while unserved:
        if idx_v >= len(vehicles):
            if len(vehicles) < num_vehicles:
                # spawn new xe at thời điểm xuất hiện sớm nhất
                t0 = min(df.at[i, 'time'] for i in unserved)
                vehicles.append({
                    'route': [0],
                    'current_node': 0,
                    'time': t0,
                    'capacity': MAX_CAPACITY,
                    'distance': 0.0
                })
            else:
                break

        veh = vehicles[idx_v]
        custs = available_customers(veh['time'])
        if not custs:
            # fast‑forward đến lần xuất hiện kế
            t_next = min(df.at[i, 'time'] for i in unserved)
            total_vehicle_idle += (t_next - veh['time'])
            veh['time'] = t_next
            custs = available_customers(veh['time'])
            if not custs:
                idx_v += 1
                continue

        # chọn khách khả dụng có deadline nhỏ nhất, tie-break by distance
        best, best_metric = None, (float('inf'), float('inf'))
        for i in custs:
            row = df.iloc[i]
            if row.demand > veh['capacity']:
                continue
            prev = df.iloc[veh['current_node']]
            d = euclidean_distance(prev.x, prev.y, row.x, row.y)
            depart = veh['time'] + d/TRUCK_SPEED
            arrival = max(depart, row.open)
            if arrival > row.close:
                continue
            metric = (row.close, d)
            if metric < best_metric:
                best_metric = metric
                best = i

        if best is None:
            idx_v += 1
            continue

        # phục vụ best
        row = df.iloc[best]
        prev = df.iloc[veh['current_node']]
        d = euclidean_distance(prev.x, prev.y, row.x, row.y)
        depart = veh['time'] + d/TRUCK_SPEED
        idle = max(0.0, row.open - depart)
        total_vehicle_idle += idle
        arrival = max(depart, row.open)
        customer_wait = max(0.0, arrival - row.time)
        total_customer_wait += customer_wait

        veh['time'] = arrival + SERVICE_TIME
        veh['capacity'] -= row.demand
        veh['distance'] += d
        veh['current_node'] = best
        veh['route'].append(best)

        unserved.remove(best)
        served.add(best)

    # quay về depot
    for veh in vehicles:
        if veh['current_node'] != 0:
            last = df.iloc[veh['current_node']]
            depot = df.iloc[0]
            d0 = euclidean_distance(last.x, last.y, depot.x, depot.y)
            veh['distance'] += d0
            veh['route'].append(0)

    return vehicles, total_vehicle_idle, total_customer_wait

# ======================
# Main: chạy thử k=6..10
# ======================
if __name__=="__main__":
    # đọc file (giả sử file có cột x,y,demand,open,close,time)
    df = pd.read_csv("F:/KLTN/100_Customer/h100c101.csv")
    for col in ['x','y','demand','open','close','time']:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    for k in range(6, 11):
        vehs, v_idle, c_wait = greedy_dscvrptw(df, num_vehicles=k)
        total_dist = sum(v['distance'] for v in vehs)
        print(f"\n--- k={k}: dùng {len(vehs)} xe ---")
        print(f"Total distance      = {total_dist:.2f}")
        print(f"Vehicle total idle  = {v_idle:.2f}")
        print(f"Customer total wait = {c_wait:.2f}")
        for i,v in enumerate(vehs):
            print(f"  Xe{i+1}: route={v['route']}, dist={v['distance']:.2f}")



--- k=6: dùng 6 xe ---
Total distance      = 2941.41
Vehicle total idle  = 1231.45
Customer total wait = 30164.99
  Xe1: route=[0, 2, 4, 13, 8, 5, 24, 25, 12, 45, 50, 54, 51, 41, 35, 79, 72, 32, 58, 81, 49, 76, 11, 69, 9, 83, 18, 96, 66, 22, 0], dist=669.31
  Xe2: route=[0, 20, 21, 3, 27, 7, 46, 61, 59, 28, 48, 68, 77, 73, 74, 14, 47, 62, 86, 94, 92, 65, 43, 19, 98, 95, 75, 80, 34, 93, 99, 0], dist=795.75
  Xe3: route=[0, 10, 15, 17, 33, 31, 37, 64, 60, 42, 71, 39, 40, 63, 16, 84, 82, 23, 55, 90, 1, 53, 91, 57, 97, 100, 0], dist=740.43
  Xe4: route=[0, 6, 85, 89, 0], dist=179.08
  Xe5: route=[0, 26, 29, 44, 78, 38, 87, 88, 0], dist=369.17
  Xe6: route=[0, 30, 52, 67, 0], dist=187.68

--- k=7: dùng 7 xe ---
Total distance      = 3047.68
Vehicle total idle  = 1483.54
Customer total wait = 30486.67
  Xe1: route=[0, 2, 4, 13, 8, 5, 24, 25, 12, 45, 50, 54, 51, 41, 35, 79, 72, 32, 58, 81, 49, 76, 11, 69, 9, 83, 18, 96, 66, 22, 0], dist=669.31
  Xe2: route=[0, 20, 21, 3, 27, 7, 46, 61, 59, 2